# Dynamic Programming (Part 2!)

In this notebook, we'll implement the Dynamic Programming algorithms for Edit Distance and Traveling Salesperson that we saw in class. 

### If you're using Datahub:
* Run the cell below **and restart the kernel if needed**

### If you're running locally:
You'll need to perform some extra setup.
#### First-time setup
* Install Anaconda following the instructions here: https://www.anaconda.com/products/distribution 
* Create a conda environment: `conda create -n cs170 python=3.11`
* Activate the environment: `conda activate cs170`
    * See for more details on creating conda environments https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html
* Install pip: `conda install pip`
* Install jupyter: `conda install jupyter`

#### Every time you want to work
* Make sure you've activated the conda environment: `conda activate cs170`
* Launch jupyter: `jupyter notebook` or `jupyter lab` 
* Run the cell below **and restart the kernel if needed**

In [2]:
# Install dependencies
!pip install -r requirements.txt --quiet

In [4]:
import otter
assert (otter.__version__ >= "5.5.0"), "Please reinstall the requirements and restart your kernel."

grader = otter.Notebook("hw07.ipynb")
import numpy.random as random
from networkx import Graph, draw
import string
import pylev
import tqdm
import time
import pickle 

from autograder_utils import validate_tour, handle_timeout, profile

test_data = pickle.load(open("public_data.pkl", "rb"))

rng_seed = 0

## Q1: Edit Distance (Global Alignment)

The edit distance problem finds the minimal number of insertions, deletions and substitutions of characters required to transform one word into another. A big application of this problem is finding the global alignment between two strings, which is often used in computational biology. 

As described in the textbook https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=6.

> A natural measure of the distance between two strings is the extent to which they can be aligned, or matched up. Technically, an alignment is simply a way of writing the strings one above the other. For instance, here are two possible alignments of SNOWY and SUNNY:

>    S−NOWY  |  −SNOW−Y <br />
>    SUNN−Y   &nbsp;|  SUN−−NY <br />
>    Cost: 3  &nbsp;&nbsp;&nbsp;&nbsp;| Cost: 5

>The “−” indicates a “gap”; any number of these can be placed in either string. The cost of an alignment is the number of columns in which the letters differ. And the edit distance between two strings is the cost of their best possible alignment. 

In this problem, you will implement an algorithm to compute the alignment between two strings $x$ and $y$, specifically, your algorithm should return the global alignment (as shown above), not just an integer value denoting the edit distance. 

### Q1.0 (Optional): The following section will walk you through how to implement this algorithm.  
**This section contains ungraded multiple choice questions to test your understanding. If you like, you may skip to Q1.1 which is the only graded question.**

Inputs: 
- x:string = length n string
- y:string = length m string

Algorithm Sketch:
1. Compute the dp subproblems as described in class and the textbook
2. Using the memoized subproblems from step 1, reconstructing the optimal global alignment 


Step 1 can be computed by simply implementing the pseudocode described in the textbook. 

Step 2 can be computed using an approach called backtracking which we walk through here. Recall that all DP have underlying DAG's where nodes represent subproblems. See https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=9. On this DAG, the DP algorithm finds the shortest path from `(0,0)` to `(n,m)`. The length of the shortest path is our edit distance, and the edges in the path correspond to the global alignment. In our back tracking algorithm, we start at `(n,m)` and reconstruct the shortest path to `(0,0)`. Since we start with `(n,m)` and end at `(0,0)`, we are back tracking the computations we did in step 1, hence the name. 

__Sanity Check (ungraded):__
Suppose we computed the DP matrix on the strings `x` and `y` want to find the edit distance between the first `5` characters of `x` and the first `6` characters of `y`. On the underlying DAG, this corresponds to the shortest path from `(0,0)` to which node? Give your answer as a tuple containing 2 integers.

In [33]:
node = (5,6)

In [34]:
grader.check("s1 (optional)")

s1 (optional) results: All test cases passed!

Now suppose that we have a way to reconstruct this shortest path, we need to convert the edges on this path into the actual alignment. 

__Sanity Check (ungraded):__ Suppose that our algorithm backtracks to node `(i,j)` and determines that the edge `(i-1,j)->(i,j)` is in this shortest path. So far, the algorithm computed 2 strings `x_align` and `y_align` based on the path from `(i,j)` to `(n,m)`. These correspond to an alignment of the substrings `x[i:n]` and `y[j:n]`. Given this new edge, what characters should you add to `x_align` and `y_align`? Input your answer choice as list of ints (ie `ans = [1]` or `ans = [1,2]`), where each int represents one of the following choices:

1. add a gap to the start of `x_align`
2. add a gap to the start of `y_align`
3. add `x[i-1]` to the start of `x_align`
4. add `y[j-1]` to the start of `y_align`

*Hint: a character must be added to both strings since at each step, `len(x_align) == len(y_align)`.*

In [43]:
ans = [3, 2]

In [44]:
grader.check("s2 (optional)")

Correct, an edge (i-1,j)->(i,j) corresponds to deleting a character from x in our original edit distance. In terms of global alignment, this is equivalent to adding a gap to y and having that gap correspond to the next character in x which is x[i-1]


s2 (optional) results: All test cases passed!

__Sanity Check (ungraded):__ Suppose that our algorithm backtracks to node `(i,j)` and determines that the edge `(i,j-1)->(i,j)` is in this shortest path. So far, the algorithm computed 2 strings `x_align` and `y_align` based on the path from `(i,j)` to `(n,m)`. These correspond to an alignment of the substrings `x[i:n]` and `y[j:n]`. Given this new edge, what characters should you add to `x_align` and `y_align`? Input your answer choice as list of ints (ie `ans = [1]` or `ans = [1,2]`), where each int represents one of the following choices:

1. add a gap to the start of `x_align`
2. add a gap to the start of `y_align`
3. add `x[i-1]` to the start of `x_align`
4. add `y[j-1]` to the start of `y_align`

*Hint: a character must be added to both strings since at each step, `len(x_align) == len(y_align)`.*

In [45]:
ans = [1, 4]

In [46]:
grader.check("s3 (optional)")

Correct, an edge (i,j-1)->(i,j) corresponds to inserting a character int x in our original edit distance. In terms of global alignment, this is equivalent to adding a gap to x and having that gap correspond to the next character in y which is y[j-1]


s3 (optional) results: All test cases passed!

__Sanity Check (ungraded):__ Suppose that our algorithm backtracks to node `(i,j)` and determines that the edge `(i-1,j-1)->(i,j)` is in this shortest path. So far, the algorithm computed 2 strings `x_align` and `y_align` based on the path from `(i,j)` to `(n,m)`. These correspond to an alignment of the substrings `x[i:n]` and `y[j:n]`. Given this new edge, what characters should you add to `x_align` and `y_align`? Input your answer choice as list of ints (ie `ans = [1]` or `ans = [1,2]`), where each int represents one of the following four choices:

1. add a gap to the start of `x_align`
2. add a gap to the start of `y_align`
3. add `x[i-1]` to the start of `x_align`
4. add `y[j-1]` to the start of `y_align`

*Hint: a character must be added to both strings since at each step, `len(x_align) == len(y_align)`.*

In [47]:
ans = [3, 4]

In [48]:
grader.check("s4 (optional)")

Correct, an edge (i-1,j-1)->(i,j) corresponds to substituting a character int x in our original edit distance. In terms of global alignment, this is equivalent to adding the next characters of both x and y.


s4 (optional) results: All test cases passed!

Since we know how to translate edges into global alignment, we now want to reconstruct the actual path from `(n,m)` to `(0,0)`. If an edge `(a,b)->(c,d)` is part of the shortest path, this means the subproblem `(a,b)` was used to compute the solution to `(c,d)`. For the edit distance problem, we know that the subproblem `(i,j)` is computed from the either `(i-1,j)`, `(i,j-1)`, or `(i-1,j-1)`. Therefore, if the node `(i,j)` is visited in the shortest path, then one of the edges `(i-1,j)->(i,j)`, `(i,j-1)->(i,j)`, or `(i-1,j-1)->(i,j)` is in the shortest path. 

We can figure out the correct edge based on the values in the dp matrix. Recall the recurrence of edit distance: `dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + diff(i,j))`. This means that at least one of the three values `dp[i - 1][j] + 1`, `dp[i][j - 1] + 1`, or `dp[i - 1][j - 1] + diff(i,j)` must equal `dp[i][j]`. If the value equals `dp[i][j]`, then that is a possible previous subproblem; otherwise, it is not. If there are multiple possible previous problems, you may back track to any one of them. 

__Sanity Check (ungraded):__ Suppose you know that `dp[i][j] = 5` and following values in the DP matrix. Which subproblems could be used to compute `dp[i][j]`? Input your answer choice as list of ints (ie `ans = [1]` or `ans = [1,2]`)
1. `dp[i-1][j] = 4`
2. `dp[i][j-1] = 5`
3. `dp[i-1][j-1] = 5, diff(i,j) = 0`

In [12]:
ans = 

In [13]:
grader.check("s5 (optional)")

s5 (optional) results:
    s5 (optional) - 1 result:
        ❌ Test case failed
        Error at line 10 in test s5 (optional):
              assert sorted(ans) == [1,3], f"Try again, {ans} is not the correct answer."
        TypeError: ellipsis' object is not iterable

__Sanity Check (ungraded):__ Suppose you know that `dp[i][j] = 9` and following values in the DP matrix. Which subproblems could be used to compute `dp[i][j]`? Input your answer choice as list of ints (ie `ans = [1]` or `ans = [1,2]`)
1. `dp[i-1][j] = 9`
2. `dp[i][j-1] = 8`
3. `dp[i-1][j-1] = 9, diff(i,j) = 1`

In [14]:
ans = ...

In [15]:
grader.check("s6 (optional)")

s6 (optional) results:
    s6 (optional) - 1 result:
        ❌ Test case failed
        Error at line 10 in test s6 (optional):
              assert sorted(ans) == [2], f"Try again, {ans} is not the correct answer."
        TypeError: ellipsis' object is not iterable

Following this logic, we start at `(n,m)` and repeatedly find the previous node until we reach `(0,0)`. Each time we backtrack one step, we update the alignment based on the edge we took.

### Q1.1 Edit Distance (Graded)
Now, implement the `edit_distance` function itself! This is the only part of Q1 that will be graded.

In [5]:
def edit_distance(x, y):
    """
    args:
        x:string = the first word.
        y:string = The second word.
    
    return:
        Tuple[String,String] = the optimum global alignment between x and y. The first string in the 
        tuple corresponds to x and the second to y. Use hypen's '-' to represent gaps in each string.
    """
    n = len(x)
    m = len(y)
    f = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    for i in range(n + 1):
        f[i][0] = i
    for j in range(m + 1):
        f[0][j] = j


    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if x[i - 1] == y[j - 1]:
                f[i][j] = f[i - 1][j - 1]
            else:
                f[i][j] = min(f[i - 1][j - 1], f[i - 1][j], f[i][j - 1]) + 1

    i = n
    j = m
    editted_x, editted_y = [], []
    while i > 0 or j > 0:
        if i > 0 and j > 0 and x[i - 1] == y[j - 1]:
            editted_x.append(x[i - 1])
            editted_y.append(y[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and j > 0 and f[i][j] == f[i - 1][j - 1] + 1:
            editted_x.append(x[i - 1])
            editted_y.append(y[j - 1])
            i -= 1
            j -= 1
        elif i > 0 and f[i][j] == f[i - 1][j] + 1:
            editted_x.append(x[i - 1])
            editted_y.append("-")
            i -= 1
        else:
            editted_x.append("-")
            editted_y.append(y[j - 1])
            j -= 1
    
    return (editted_x[::-1], editted_y[::-1])

In [17]:
grader.check("edit_distance")

100%|██████████| 200/200 [00:25<00:00,  7.92it/s]


edit_distance results: All test cases passed!

### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

You can roughly split this task into two parts: finding the optimal edit distance and then reconstructing the best possible alignment. When debugging, might want to first ensure that your edit distances are correct before checking your alignment code. Additionally, when debugging the alignment, you might want to consider working through some small examples by hand - the examples from the textbook are a great place to start, as well as any examples that might invoke particular edge cases.

In [18]:
rng = random.default_rng(rng_seed)

start = time.time()
def check_word(original, aligned):
    ''' checks that the string `aligned` is obtained by only adding gaps to the string `original`'''
    
    assert len(aligned) >= len(original), "your function returned a string which is shorter than the input string!"
    i,j = 0,0
    while i < len(original) and j < len(aligned):
        while aligned[j] == '-' and j < len(aligned):
            j += 1
        assert original[i] == aligned[j], "your function returned a string which cannot be produced by only adding gaps!"
        i += 1
        j += 1
    while j < len(aligned):
        assert aligned[j] == '-', "your function returned a string which cannot be produced by only adding gaps!"
        j += 1
        
NUM_TRIALS = 200
LETTERS = list(string.ascii_lowercase)
MIN_WORD_SIZE = 250
MAX_WORD_SIZE = 500

for i in tqdm.tqdm(range(NUM_TRIALS)):
    word1_size, word2_size = rng.integers(MIN_WORD_SIZE, MAX_WORD_SIZE, size=2)
    word1 = ''.join(rng.choice(LETTERS, size=word1_size))
    word2 = ''.join(rng.choice(LETTERS, size=word2_size))
    align1,align2 = edit_distance(word1, word2)
    
    assert len(align1) == len(align2), f"""a global alignment requires the two strings to be the same 
        length, your functions returns two strings of length {len(align1)} and {len(align2)}!"""
    
    check_word(word1, align1)
    check_word(word2, align2)
    
    dist = 0
    for a,b in zip(align1,align2):
        if a != b:
            dist += 1
    staff_distance = pylev.levenshtein(word1, word2)
    assert staff_distance == dist, f"""the inputs have an edit distance of {staff_distance}, but your
        strings have a distance of {dist}."""
finish = time.time()
assert finish - start < 60, "your solution timed out"

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:25<00:00,  7.71it/s]


## Q2) Traveling Salesperson DP
Now, we'll implement the dynamic programming algorithm for the traveling salesperson problem (TSP). A brute force solution will be hopelessly slow even for moderate-sized test cases, but we can use dynamic programming to get a solution in slightly more reasonable (but still exponential) time. For a refresher on the TSP algorithm, see Lecture 12 or https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf#page=20. 

As with previous problems, we want you to return the actual tour, not the cost of the tour. We can once again apply the same procedure of backtracking through our subproblem array to reconstruct this tour.

### Representing Subproblems
If we use a set as one of our subproblem parameters, we can't directly use a 2D array to store our subproblems. There are two common ways to work around this issue:

#### 1. Subproblem Dictionary
You could store subproblems in a dictionary, where the keys are tuples of the form `(S, i)`, where `i` represents the last city visited before returning home and `S` is the set of cities visited so far. 

To make this work, you need to ensure that the keys are hashable. One way is using Python's built-in `frozenset` class for `S`. `frozenset` is a built-in type so you can use it without any additional imports, and works just like a normal set, except that it is immutable (and hashable). You can read more about `frozenset` here: https://docs.python.org/3/library/stdtypes.html#frozenset.

#### 2. Bitmasking
Instead of a hash set, we actually *can* still use a 2D array to store subproblems, where `S` is represented as an $n$-bit unsigned integer, and the $i$-th bit of `S` would be set to 1 if and only if the $i$-th city is part of the set of visited cities. Since `S` is an integer, we can use it to index into our 2D array.

The bitmasking approach tends to be about twice as fast and much more memory-efficient than the `frozenset` approach, but both approaches will pass the autograder if implemented correctly.

### Implementation tip:
As with before, storing the entire tour at each step is too memory-intensive and will cause the autograder to fail. Instead, consider maintaining a separate dictionary or array which stores a smaller amount of information but can still help you reconstruct the tour (can the "shortest path in the DP DAG" idea help here?).

**Be careful with indexing!** The algorithm from the book assumes your cities are labeled $1, \dots, n$ - if you are indexing into a Python list, will you need to adjust your indices?

Be careful with subproblem ordering! We need to ensure that whenever we go to solve a subproblem, all of the subproblems it depends on have already been solved.

The graph is not necessarily complete! If no tour is possible, return an empty list.

### Graph helpers
Like the last homeworks, we use a weighted adjacency list to represent the graph. We'll use a similar format as before, except `graph[u]` is now a hashmap instead of a list of pairs. 
 **For this assignment, graphs are undirected**, so if there is an (undirected) edge between nodes `u` and `v` with weight `w`, then `graph[u]` contains key `v` with value `w` and `graph[v]` contains key `u` with value `w`.

We provide the following code to help you test your implementation.

In [6]:
def generate_adj_list(n, edge_list: list[tuple[int]]) -> list[dict[int, int]]:
    """
    args:
        n:int = number of nodes in the graph. The nodes are labelled with integers 0 through n-1
        edge_list:list[tuple[int,int,int]] = edge list where each tuple (u,v,w) represents the 
        undirected and weighted edge (u,v,w) in the graph
    return:
        A list[dict[int, int]] representing the adjacency list 
    """
    adj_list = [{} for _ in range(n)] 
    for u, v, w in sorted(edge_list):
        adj_list[u][v] = w
        # undirected edges
        adj_list[v][u] = w
    return adj_list

def draw_graph(adj_list: list[dict[int, int]]):
    """Utility method for visualizing graphs

    args:
        adj_list: list[dict[int, int]] = adjacency list representation of the graph generated by generate_adj_list
    """
    G = Graph()
    for u in range(len(adj_list)):
        for v, w in adj_list[u]:
            G.add_edge(u, v, weight=w)
    draw(G, with_labels=True)

In [20]:
edge = [(0, 1, 1), (0, 2, 2), (1, 2, 3), (1, 3, 4), (2, 3, 5)]

adj = generate_adj_list(4, edge)
# draw_graph(adj)

print(adj)

[{1: 1, 2: 2}, {0: 1, 2: 3, 3: 4}, {0: 2, 1: 3, 3: 5}, {1: 4, 2: 5}]


In [28]:
def tsp_dp(adj_list):
    """Compute the exact solution to the TSP using dynamic programming and returns the optimal path.

    Args:
        dist_arr: Weighted undirected graph represented as an adjacency list. 

    Returns:
        List[int]: A list of city indices representing the optimal path.
    """
    # ...
    def get_weights(n, adj_list):
        weights = [[-1 for _ in range(n)] for _ in range(n)]
        for u, edges in enumerate(adj_list):
            for v, w in edges.items():
                weights[u][v] = w
                weights[v][u] = w
        return weights

    n = len(adj_list)
    for dict in adj_list:
        n = max(n, max(dict.keys()) + 1)

    # print(adj_list)
    weights = get_weights(n, adj_list)

    f = [[-1 for _ in range(n)] for _ in range(1 << n)]
    f[1][0] = 0
    path = [[-1 for _ in range(n)] for _ in range(1 << n)]
    vis = [[False for _ in range(n)] for _ in range(1 << n)]
    vis[1][0] = True

    def dfs(sta, u):
        if vis[sta][u] or sta & 1 == 0:
            return f[sta][u]
        
        # print(f"In state {bin(sta)}, at node {u}")
        vis[sta][u] = True

        '''
        The state get value from a state that not visit u, use a path v->u
        '''
        mask = sta & ~(1 << u)
        # print(bin(mask))
        for v in range(n):
            if (mask & (1 << v)) and weights[u][v] != -1:
                val = dfs(mask, v)
                if val == -1:
                    continue
                val += weights[u][v]
                # print(f"from {bin(sta)}, {u} search {bin(mask)}, {v}, get {val}")
                if f[sta][u] == -1 or val < f[sta][u]:
                    
                    f[sta][u] = val
                    path[sta][u] = v

        return f[sta][u]
    
    last = None
    for i in range(1, n):
        if weights[0][i] == -1:
            continue
        val = dfs((1 << n) - 1, i)
        if val == -1:
            continue
        val += weights[i][0]
        if last == None or val < last:
            last = val
            path[(1 << n) - 1][0] = i
            f[(1 << n) - 1][0] = val

    # for sta in range(1 << n):
    #     for i in range(n):
    #         if f[sta][i] == -1:
    #             continue
    #         print(f"{bin(sta)} {i} {f[sta][i]}")

    if last == None:
        return []

    ans = [0]
    sta = (1 << n) - 1
    u = path[sta][0]
    # print(u)
    while u != 0:
        ans.append(u)
        v = path[sta][u]
        sta = sta & ~(1 << u)
        u = v

    ans.reverse()

    assert len(ans) >= 0, "???"
    return ans

In [22]:
adj = [{3: 5, 4: 5}, {2: 5, 3: 4, 4: 9}, {1: 5, 4: 4}, {0: 5, 1: 4}, {0: 5, 1: 9, 2: 4}]

print(tsp_dp(adj))

[{3: 5, 4: 5}, {2: 5, 3: 4, 4: 9}, {1: 5, 4: 4}, {0: 5, 1: 4}, {0: 5, 1: 9, 2: 4}]
[0, 4, 2, 1, 3, 0]


In [29]:
grader.check("TSP")

100%|██████████| 16/16 [00:04<00:00,  3.71it/s]


TSP results: All test cases passed!

### Debugging
A simplified verion of the otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

In [30]:
# tests on very small cases
for adj_list, expected_distance in tqdm.tqdm(test_data['TSP-1']):
    result = tsp_dp(adj_list)

    if expected_distance < 0: 
        # no tour is possible
        assert result == [], "You returned a tour when no tour is possible"
    else:
        assert set(result) == set(range(len(adj_list))), f"Your output does not visit all cities"
        student_length = validate_tour(result, adj_list)
        assert student_length >= 0, f"Your output is not a valid tour, {result}"
        assert student_length == expected_distance, f"Your output is not a minimum distance tour, expected {expected_distance} but got {student_length}"

100%|██████████| 20/20 [00:05<00:00,  3.78it/s]


## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [31]:
grader.export(pdf=False, force_save=True, run_tests=True)

<IPython.core.display.Javascript object>